# 🏥 Medical Image Segmentation - Google Colab Training

**Project:** Comparing UNet, UNet++, and TransUNet on ISIC Dataset

**Author:** Prabhat

---

## 📋 Training Plan

We'll train 3 models × 4 data fractions = **12 experiments**

| Model | 10% | 25% | 50% | 100% |
|-------|-----|-----|-----|------|
| UNet | ✓ | ✓ | ✓ | ✓ |
| UNet++ | ✓ | ✓ | ✓ | ✓ |
| TransUNet | ✓ | ✓ | ✓ | ✓ |

**Estimated Time:** 6-8 hours total

## 1️⃣ Setup: Check GPU

In [ ]:
# Check GPU availability
!nvidia-smi

Thu Dec  4 16:39:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ WARNING: No GPU detected! Training will be very slow.")
    print("Go to Runtime → Change runtime type → Select GPU")

PyTorch version: 2.9.0+cu126
CUDA available: True
CUDA version: 12.6
GPU: Tesla T4
GPU Memory: 15.83 GB


## 2️⃣ Clone Repository

In [ ]:
# Clone your GitHub repository
!git clone https://github.com/Prabhat9801/Medical-Image-Segmentation.git
%cd Medical-Image-Segmentation

# Verify
!ls -la

Cloning into 'Medical-Image-Segmentation'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 48 (delta 11), reused 42 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (48/48), 56.56 KiB | 981.00 KiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/Medical-Image-Segmentation
total 120
drwxr-xr-x 6 root root 4096 Dec  4 16:40 .
drwxr-xr-x 1 root root 4096 Dec  4 16:40 ..
-rw-r--r-- 1 root root 5337 Dec  4 16:40 CHECKLIST.md
-rw-r--r-- 1 root root 4365 Dec  4 16:40 DATASET_DOWNLOAD.md
-rw-r--r-- 1 root root 6365 Dec  4 16:40 download_isic_dataset.py
drwxr-xr-x 8 root root 4096 Dec  4 16:40 .git
-rw-r--r-- 1 root root 5016 Dec  4 16:40 .gitignore
-rw-r--r-- 1 root root 1076 Dec  4 16:40 LICENSE
drwxr-xr-x 2 root root 4096 Dec  4 16:40 notebooks
-rw-r--r-- 1 root root 9791 Dec  4 16:40 PROJECT_SUMMARY.md
-rw-r--r-- 1 root root 6016 Dec  4 16:40 QUICKSTART.md
-rw-r--r-- 1 root 

## 3️⃣ Install Dependencies

In [ ]:
# Install required packages
!pip install -q timm albumentations opencv-python-headless tqdm

print("✅ Dependencies installed!")

✅ Dependencies installed!


## 4️⃣ Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Verify your zip file is there
!ls -lh /content/drive/MyDrive/*.zip

Mounted at /content/drive
-rw------- 1 root root 222M Dec  4 14:49 /content/drive/MyDrive/isic_processed_256.zip


## 5️⃣ Extract Processed Data

In [ ]:
import zipfile
import os
import shutil
import pandas as pd

# Paths
zip_path = "/content/drive/MyDrive/isic_processed_256.zip"
final_path = "/content/Medical-Image-Segmentation/data/processed/isic"

print("📦 Extracting data with Windows path fix...")

# Remove existing if present
if os.path.exists(final_path):
    shutil.rmtree(final_path)

# Create the directory structure
os.makedirs(final_path, exist_ok=True)
os.makedirs(os.path.join(final_path, "images"), exist_ok=True)
os.makedirs(os.path.join(final_path, "masks"), exist_ok=True)

# Extract and fix Windows paths
with zipfile.ZipFile(zip_path, 'r') as z:
    for file_info in z.filelist:
        # Get the original filename
        orig_name = file_info.filename

        # Convert Windows backslashes to forward slashes
        fixed_name = orig_name.replace('\\', '/')

        # Remove 'isic/' prefix if present
        if fixed_name.startswith('isic/'):
            fixed_name = fixed_name[5:]  # Remove 'isic/' prefix

        # Determine target path
        if 'images/' in fixed_name or (fixed_name.endswith('.png') and 'mask' not in fixed_name.lower()):
            # It's an image
            filename = os.path.basename(fixed_name)
            target_path = os.path.join(final_path, "images", filename)
        elif 'masks/' in fixed_name or '_mask.png' in fixed_name:
            # It's a mask
            filename = os.path.basename(fixed_name)
            target_path = os.path.join(final_path, "masks", filename)
        elif fixed_name.endswith('splits.csv'):
            # It's the splits file
            target_path = os.path.join(final_path, "splits.csv")
        else:
            # Skip other files
            continue

        # Extract the file
        with z.open(file_info) as source, open(target_path, 'wb') as target:
            shutil.copyfileobj(source, target)

print("✅ Extraction complete!")

# Fix splits.csv to use forward slashes
print("\n🔧 Fixing splits.csv paths...")
splits_path = os.path.join(final_path, "splits.csv")
if os.path.exists(splits_path):
    df = pd.read_csv(splits_path)

    # Convert backslashes to forward slashes in all columns
    for col in df.columns:
        if df[col].dtype == 'object':  # String columns
            df[col] = df[col].str.replace('\\', '/', regex=False)

    # Save the fixed CSV
    df.to_csv(splits_path, index=False)
    print("✅ splits.csv paths fixed!")
else:
    print("❌ splits.csv not found!")

# Verify
print("\n" + "="*60)
print("📊 Verification:")
print("="*60)

import glob
image_count = len(glob.glob(os.path.join(final_path, "images", "*.png")))
mask_count = len(glob.glob(os.path.join(final_path, "masks", "*.png")))
splits_exists = os.path.exists(splits_path)

print(f"✅ Images: {image_count}")
print(f"✅ Masks: {mask_count}")
print(f"✅ splits.csv: {'Found' if splits_exists else 'NOT FOUND'}")

# Show sample from splits.csv
if splits_exists:
    df_check = pd.read_csv(splits_path)
    print(f"\n📄 Sample from splits.csv:")
    print(df_check.head(3))
    print(f"\nTotal rows: {len(df_check)}")

if image_count > 0 and mask_count > 0 and splits_exists:
    print("\n🎉 Data extraction successful!")
else:
    print("\n❌ Something went wrong!")

📦 Extracting data with Windows path fix...
✅ Extraction complete!

🔧 Fixing splits.csv paths...
✅ splits.csv paths fixed!

📊 Verification:
✅ Images: 2594
✅ Masks: 2594
✅ splits.csv: Found

📄 Sample from splits.csv:
                                       image_path  \
0  ../data/processed/isic/images/ISIC_0016024.png   
1  ../data/processed/isic/images/ISIC_0000026.png   
2  ../data/processed/isic/images/ISIC_0000175.png   

                                           mask_path  split  
0  ../data/processed/isic/masks/ISIC_0016024_mask...  train  
1  ../data/processed/isic/masks/ISIC_0000026_mask...  train  
2  ../data/processed/isic/masks/ISIC_0000175_mask...  train  

Total rows: 2594

🎉 Data extraction successful!


In [ ]:
import os

# Check if the file exists
test_file = "/content/Medical-Image-Segmentation/data/processed/isic/images/ISIC_0014365.png"
print(f"File exists: {os.path.exists(test_file)}")

# List some files
data_dir = "/content/Medical-Image-Segmentation/data/processed/isic"
if os.path.exists(data_dir):
    print(f"\n📁 Contents of {data_dir}:")
    print(os.listdir(data_dir))

    images_dir = os.path.join(data_dir, "images")
    if os.path.exists(images_dir):
        image_files = os.listdir(images_dir)
        print(f"\n📷 Found {len(image_files)} images")
        print(f"Sample: {image_files[:5]}")

File exists: True

📁 Contents of /content/Medical-Image-Segmentation/data/processed/isic:
['images', 'masks', 'splits.csv']

📷 Found 2594 images
Sample: ['ISIC_0014554.png', 'ISIC_0015482.png', 'ISIC_0010468.png', 'ISIC_0013765.png', 'ISIC_0012313.png']


In [ ]:
import pandas as pd
import os
from pathlib import Path

# Paths
data_dir = "/content/Medical-Image-Segmentation/data/processed/isic"
splits_path = os.path.join(data_dir, "splits.csv")

print("🔧 Fixing splits.csv paths for Colab...")

# Read the current splits.csv
df = pd.read_csv(splits_path)

print(f"Original paths (first row):")
print(f"  Image: {df.iloc[0]['image_path']}")
print(f"  Mask: {df.iloc[0]['mask_path']}")

# Fix the paths - extract just the filename and rebuild with correct base path
df['image_path'] = df['image_path'].apply(
    lambda x: os.path.join(data_dir, "images", os.path.basename(x))
)
df['mask_path'] = df['mask_path'].apply(
    lambda x: os.path.join(data_dir, "masks", os.path.basename(x))
)

print(f"\nFixed paths (first row):")
print(f"  Image: {df.iloc[0]['image_path']}")
print(f"  Mask: {df.iloc[0]['mask_path']}")

# Verify the files exist
sample_image = df.iloc[0]['image_path']
sample_mask = df.iloc[0]['mask_path']
print(f"\n✅ Sample image exists: {os.path.exists(sample_image)}")
print(f"✅ Sample mask exists: {os.path.exists(sample_mask)}")

# Save the fixed CSV
df.to_csv(splits_path, index=False)
print(f"\n✅ splits.csv updated with {len(df)} rows!")

# Show summary
print(f"\nSplit distribution:")
print(df['split'].value_counts())

🔧 Fixing splits.csv paths for Colab...
Original paths (first row):
  Image: ../data/processed/isic/images/ISIC_0016024.png
  Mask: ../data/processed/isic/masks/ISIC_0016024_mask.png

Fixed paths (first row):
  Image: /content/Medical-Image-Segmentation/data/processed/isic/images/ISIC_0016024.png
  Mask: /content/Medical-Image-Segmentation/data/processed/isic/masks/ISIC_0016024_mask.png

✅ Sample image exists: True
✅ Sample mask exists: True

✅ splits.csv updated with 2594 rows!

Split distribution:
split
train    1815
test      390
val       389
Name: count, dtype: int64


## 6️⃣ Test Run (Quick Verification)

In [ ]:
# Quick test with UNet, 10% data, 2 epochs
!python -m src.train \
    --model unet \
    --epochs 2 \
    --batch_size 8 \
    --data_fraction 0.1 \
    --lr 1e-4

print("\n✅ Test run complete! If this worked, proceed with full training.")

Using device: cuda
Experiment directory: experiments/unet_10pct_20251204_172416

Creating dataloaders...
/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/content/Medical-Image-Segmentation/src/datasets/isic_dataset.py:110: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  A.ElasticTransform(p=1.0, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
/content/Medical-Image-Segmentation/src/datasets/isic_dataset.py:112: UserWarning: Argument(s) 'shift_limit' are not valid for transform OpticalDistortion
  A.OpticalDistortion(distort_limit=1, shift_limit=0.5, p=1.0),
Loaded 181 samples for train split (fraction: 0.1)
Loaded 389 samples for val split (fraction: 1.0)
Loaded 390 samples for test split (fraction: 1.0)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/data

## 7️⃣ Full Training - UNet (All Data Fractions)

In [ ]:
# UNet - 10% data (~15-20 minutes)
!python -m Medical-Image-Segmentation.src.train \
    --model unet \
    --epochs 30 \
    --batch_size 8 \
    --data_fraction 0.1 \
    --lr 1e-4

/usr/bin/python3: Error while finding module specification for 'Medical-Image-Segmentation.src.train' (ModuleNotFoundError: No module named 'Medical-Image-Segmentation')


In [ ]:
# UNet - 25% data (~30-40 minutes)
!python -m src.train \
    --model unet \
    --epochs 30 \
    --batch_size 8 \
    --data_fraction 0.25 \
    --lr 1e-4

Using device: cuda
Experiment directory: experiments/unet_25pct_20251204_172520

Creating dataloaders...
/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/content/Medical-Image-Segmentation/src/datasets/isic_dataset.py:110: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  A.ElasticTransform(p=1.0, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
/content/Medical-Image-Segmentation/src/datasets/isic_dataset.py:112: UserWarning: Argument(s) 'shift_limit' are not valid for transform OpticalDistortion
  A.OpticalDistortion(distort_limit=1, shift_limit=0.5, p=1.0),
Loaded 453 samples for train split (fraction: 0.25)
Loaded 389 samples for val split (fraction: 1.0)
Loaded 390 samples for test split (fraction: 1.0)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dat

In [ ]:
# UNet - 50% data (~1 hour)
!python -m src.train \
    --model unet \
    --epochs 30 \
    --batch_size 8 \
    --data_fraction 0.5 \
    --lr 1e-4

Using device: cuda
Experiment directory: experiments/unet_50pct_20251204_173807

Creating dataloaders...
/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/content/Medical-Image-Segmentation/src/datasets/isic_dataset.py:110: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  A.ElasticTransform(p=1.0, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
/content/Medical-Image-Segmentation/src/datasets/isic_dataset.py:112: UserWarning: Argument(s) 'shift_limit' are not valid for transform OpticalDistortion
  A.OpticalDistortion(distort_limit=1, shift_limit=0.5, p=1.0),
Loaded 907 samples for train split (fraction: 0.5)
Loaded 389 samples for val split (fraction: 1.0)
Loaded 390 samples for test split (fraction: 1.0)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/data

In [ ]:
# UNet - 100% data (~2 hours)
!python -m src.train \
    --model unet \
    --epochs 30 \
    --batch_size 8 \
    --data_fraction 1.0 \
    --lr 1e-4

Using device: cuda
Experiment directory: experiments/unet_100pct_20251204_180029

Creating dataloaders...
/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/content/Medical-Image-Segmentation/src/datasets/isic_dataset.py:110: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  A.ElasticTransform(p=1.0, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
/content/Medical-Image-Segmentation/src/datasets/isic_dataset.py:112: UserWarning: Argument(s) 'shift_limit' are not valid for transform OpticalDistortion
  A.OpticalDistortion(distort_limit=1, shift_limit=0.5, p=1.0),
Loaded 1815 samples for train split (fraction: 1.0)
Loaded 389 samples for val split (fraction: 1.0)
Loaded 390 samples for test split (fraction: 1.0)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/da

## 8️⃣ Full Training - UNet++ (All Data Fractions)

In [ ]:
# UNet++ - 10% data
!python -m src.train \
    --model unetpp \
    --epochs 30 \
    --batch_size 8 \
    --data_fraction 0.1 \
    --lr 1e-4

Using device: cuda
Experiment directory: experiments/unetpp_10pct_20251204_184220

Creating dataloaders...
/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/content/Medical-Image-Segmentation/src/datasets/isic_dataset.py:110: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  A.ElasticTransform(p=1.0, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
/content/Medical-Image-Segmentation/src/datasets/isic_dataset.py:112: UserWarning: Argument(s) 'shift_limit' are not valid for transform OpticalDistortion
  A.OpticalDistortion(distort_limit=1, shift_limit=0.5, p=1.0),
Loaded 181 samples for train split (fraction: 0.1)
Loaded 389 samples for val split (fraction: 1.0)
Loaded 390 samples for test split (fraction: 1.0)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/da

In [ ]:
# UNet++ - 25% data
!python -m src.train \
    --model unetpp \
    --epochs 30 \
    --batch_size 8 \
    --data_fraction 0.25 \
    --lr 1e-4

Using device: cuda
Experiment directory: experiments/unetpp_25pct_20251204_185204

Creating dataloaders...
/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/content/Medical-Image-Segmentation/src/datasets/isic_dataset.py:110: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  A.ElasticTransform(p=1.0, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
/content/Medical-Image-Segmentation/src/datasets/isic_dataset.py:112: UserWarning: Argument(s) 'shift_limit' are not valid for transform OpticalDistortion
  A.OpticalDistortion(distort_limit=1, shift_limit=0.5, p=1.0),
Loaded 453 samples for train split (fraction: 0.25)
Loaded 389 samples for val split (fraction: 1.0)
Loaded 390 samples for test split (fraction: 1.0)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/d

In [ ]:
# UNet++ - 50% data
!python -m src.train \
    --model unetpp \
    --epochs 30 \
    --batch_size 8 \
    --data_fraction 0.5 \
    --lr 1e-4

Using device: cuda
Experiment directory: experiments/unetpp_50pct_20251204_191013

Creating dataloaders...
/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/content/Medical-Image-Segmentation/src/datasets/isic_dataset.py:110: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  A.ElasticTransform(p=1.0, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
/content/Medical-Image-Segmentation/src/datasets/isic_dataset.py:112: UserWarning: Argument(s) 'shift_limit' are not valid for transform OpticalDistortion
  A.OpticalDistortion(distort_limit=1, shift_limit=0.5, p=1.0),
Loaded 907 samples for train split (fraction: 0.5)
Loaded 389 samples for val split (fraction: 1.0)
Loaded 390 samples for test split (fraction: 1.0)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/da

In [ ]:
# UNet++ - 100% data
!python -m src.train \
    --model unetpp \
    --epochs 30 \
    --batch_size 8 \
    --data_fraction 1.0 \
    --lr 1e-4

Using device: cuda
Experiment directory: experiments/unetpp_100pct_20251204_194226

Creating dataloaders...
/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/content/Medical-Image-Segmentation/src/datasets/isic_dataset.py:110: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  A.ElasticTransform(p=1.0, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
/content/Medical-Image-Segmentation/src/datasets/isic_dataset.py:112: UserWarning: Argument(s) 'shift_limit' are not valid for transform OpticalDistortion
  A.OpticalDistortion(distort_limit=1, shift_limit=0.5, p=1.0),
Loaded 1815 samples for train split (fraction: 1.0)
Loaded 389 samples for val split (fraction: 1.0)
Loaded 390 samples for test split (fraction: 1.0)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/

## 9️⃣ Full Training - TransUNet (All Data Fractions)

**Note:** TransUNet uses smaller batch size (4) due to higher memory requirements

In [ ]:
# TransUNet - 10% data (~45 min)
!python -m src.train \
    --model transunet \
    --epochs 30 \
    --batch_size 4 \
    --data_fraction 0.1 \
    --lr 1e-4

In [ ]:
# TransUNet - 25% data (~1.5 hours)
!python -m src.train \
    --model transunet \
    --epochs 30 \
    --batch_size 4 \
    --data_fraction 0.25 \
    --lr 1e-4

In [ ]:
# TransUNet - 50% data (~3 hours)
!python -m src.train \
    --model transunet \
    --epochs 30 \
    --batch_size 4 \
    --data_fraction 0.5 \
    --lr 1e-4

In [ ]:
# TransUNet - 100% data (~6 hours)
!python -m src.train \
    --model transunet \
    --epochs 30 \
    --batch_size 4 \
    --data_fraction 1.0 \
    --lr 1e-4

## 🔟 View Training Results

In [ ]:
# List all experiments
!ls -lh experiments/

In [ ]:
# View training history for a specific experiment
import json
import matplotlib.pyplot as plt

# Replace with your actual experiment folder name
exp_folder = "experiments/unet_10pct_TIMESTAMP"  # UPDATE THIS

with open(f"{exp_folder}/history.json", 'r') as f:
    history = json.load(f)

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(history['train_loss'], label='Train')
ax1.plot(history['val_loss'], label='Val')
ax1.set_title('Loss')
ax1.legend()

ax2.plot(history['train_dice'], label='Train')
ax2.plot(history['val_dice'], label='Val')
ax2.set_title('Dice Score')
ax2.legend()

plt.show()

## 1️⃣1️⃣ Evaluate Models

In [ ]:
# Evaluate UNet 10% model
# Replace with your actual experiment folder
!python -m src.eval \
    --model unet \
    --checkpoint experiments/unet_10pct_TIMESTAMP/best_model.pt \
    --num_vis 8

In [ ]:
# View evaluation results
from IPython.display import Image, display

# Display predictions
display(Image('reports/figures/best_model/predictions.png'))

## 1️⃣2️⃣ Save Results to Google Drive

In [ ]:
# Create backup folders in Drive
!mkdir -p /content/drive/MyDrive/medseg_experiments
!mkdir -p /content/drive/MyDrive/medseg_reports

# Copy all experiments
print("📦 Copying experiments to Google Drive...")
!cp -r experiments/* /content/drive/MyDrive/medseg_experiments/

# Copy reports
print("📦 Copying reports to Google Drive...")
!cp -r reports/* /content/drive/MyDrive/medseg_reports/

print("\n✅ All results saved to Google Drive!")
print("📂 Location: MyDrive/medseg_experiments/ and MyDrive/medseg_reports/")

## 1️⃣3️⃣ Create Results Summary

In [ ]:
import pandas as pd
import glob
import json

# Collect all results
results_files = glob.glob('reports/figures/*/results.json')

summary_data = []

for result_file in results_files:
    with open(result_file, 'r') as f:
        data = json.load(f)

    summary_data.append({
        'Model': data['model'],
        'Checkpoint': data['checkpoint'].split('/')[-2],
        'Dice': f"{data['metrics']['dice']['mean']:.4f}",
        'IoU': f"{data['metrics']['iou']['mean']:.4f}",
        'Accuracy': f"{data['metrics']['accuracy']['mean']:.4f}"
    })

summary_df = pd.DataFrame(summary_data)
print("\n" + "="*60)
print("📊 RESULTS SUMMARY")
print("="*60)
print(summary_df.to_string(index=False))
print("="*60)

# Save to CSV
summary_df.to_csv('results_summary.csv', index=False)
!cp results_summary.csv /content/drive/MyDrive/

print("\n✅ Summary saved to Google Drive as results_summary.csv")

## ✅ Training Complete!

### Next Steps:

1. **Download results** from Google Drive:
   - `medseg_experiments/` folder
   - `medseg_reports/` folder
   - `results_summary.csv`

2. **Complete the report** (`reports/report.md`):
   - Fill in actual performance numbers
   - Add visualizations
   - Write analysis

3. **Create presentation** using the figures

4. **Update README** with findings

---

**🎉 Congratulations on completing the training!**